In [15]:
import xlsxwriter
import timeit
from itertools import zip_longest
from nltk.tokenize import sent_tokenize

# pptx인 파일 가져오기

from common_functions import *
from regex_functions import *

import glob
import subprocess

ModuleNotFoundError: No module named 'stanza'

In [16]:
pip install stanza


The following command must be run outside of the IPython shell:

    $ pip install stanza

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [13]:
# file 다 가져오기
def get_filename_list(path, ext):
    print('Change encoding files.......\ \n')
    result = []
    for f in glob.glob(file_path + f"/*{ext}"):
        # run convmv shell script -> file normalization NFC -> NFD (한글자소분리해결)
        subprocess.run(['/usr/local/bin/convmv', '-f', 'utf-8', '-t', 'utf-8', '--nfc', '--notest', f])
        result.append(f)
    return result

root_path = '/Users/jihyeoh/Lexcode/팀 채널 - 인공지능 학습 DB 구축 채널/1. 원본DB(렉스코드)'
sub_dir = '/4.2019한국표준협회'
file_path = root_path + sub_dir

pptx_files_dict = get_filename_list(file_path, '.pptx')
print(pptx_files_dict)

Change encoding files.......\ 



NameError: name 'glob' is not defined

In [10]:
def pptx_parser_pre(pptx_file):
    # pptx 분석 하기위해 list 만들어주기
    pptx_results_pre = []
    prs = Presentation(pptx_file)
    
    # ---- pptx 추출 시작 ----
    for slide in prs.slides:
        
        # 1개의 pptx 분석시작
        for shape in slide.shapes:
            if not shape.has_text_frame:
                continue
                
            
            # 전처리 및 라인별 넣어주기
            for paragraph in shape.text_frame.paragraphs:
                paragraph.text = pptx_extra_regex(paragraph.text)

                if paragraph.text.strip() in ['.', '/', ',']:
                    continue
                pptx_results_pre.append(paragraph.text)

            # sent_tokenize 
            pptx_results = []
            for pptx_result_pre in pptx_results_pre:
                text_list = sent_tokenize(pptx_result_pre)
                for text in text_list:
                    if len(text) == 1 and text == 'E':
                        text = text + '→'
                    if text[:2] in 'ver':
                        text = '←' + text
                    pptx_results.append(text)
    pptx_results = list(filter(None, pptx_results))
    return pptx_results

In [9]:
def pptx_to_excel(pptx_files_dict):

    start = timeit.default_timer()
    
    completed_log = open(f'./4_2019한국표준협회/4_completed_log' + '.txt', "w+")
    error_log = open(f'./4_2019한국표준협회/4_error_log' + '.txt', "w+")
    i = 0
    ko_files = []
    en_files = []

    for pptx_file in pptx_files_dict:
        if pptx_file[-6] == '한':
            ko_files.append(pptx_file)
        elif pptx_file[-6] == '영':
            en_files.append(pptx_file)
    print(len(ko_files), len(en_files))
    try:
        # 짝이 맞는 순서대로 넣어주기
        ko_file, en_file = '', ''
        # 한, 영 짝수 맞추기
        for ko_file in ko_files:
            for en_file in en_files:
                if ko_file[:-6] == en_file[:-6]:
                    # ---- pptx 추출 시작 ----
                    pptx_ko = pptx_parser_pre(ko_file)
                    pptx_en = pptx_parser_pre(en_file)
                    

                    i += 1       
                    # ---- 파일명에 따라서 순서대로 한, 영 key에 filename을 value로 넣어주기 ----
                    completed_log.write(str(i) + ko_file + ' \n')
                    completed_log.write(str(i) + en_file + ' \n')
                    print(i, ko_file[115:-5])
                    print(i, en_file[115:-5])


                    # excel 밖으로 빼내기
                    # ------- 절대경로보다 상대경로 사용해주세요
                    workbook = xlsxwriter.Workbook(f'./4_2019한국표준협회/Test_3/{ko_file[115:-6]}.xlsx')
                    print(i, ko_file[115:-6])
                    worksheet = workbook.add_worksheet()
                    worksheet.write('A1', ' ')
                    worksheet.write('B1', 'KOR')
                    worksheet.write('C1', 'ENG')

                    row_idx = 2

                    # pptx을 excel로 
                    worksheet.write('A' + str(row_idx), '>' * 10 + ko_file[115:-6])
                    row_idx += 1
                    for ko, en in zip_longest(pptx_ko, pptx_en, fillvalue=' '):
                        worksheet.write('B' + str(row_idx), ko)
                        worksheet.write('C' + str(row_idx), en)
                        row_idx += 1

                    workbook.close()

    except Exception as e:
        error_log.write(str(i) + ko_file + ' \n')

    stop = timeit.default_timer()
    print('Running Time: ', stop - start)

    print(f'ko :{len(ko_files)}, en :{len(en_files)}')

In [10]:
pptx_to_excel(pptx_files_dict)

NameError: name 'pptx_files_dict' is not defined